In [1]:
import math
import random
from random import choice, randint, random

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import tqdm
from copy import deepcopy as dc

from collections import Counter
from datetime import datetime, timedelta

from sklearn.model_selection import train_test_split

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModel,
    DataCollatorWithPadding,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    pipeline,
)
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'


import evaluate
import glob


import warnings
warnings.filterwarnings("ignore")

C:\Users\Юрий\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [57]:
model_name = "blanchefort/rubert-base-cased-sentiment"  # Модель для классификации на русском
tokenizer = AutoTokenizer.from_pretrained(model_name)
print('tokenizer_done')
model = AutoModelForSequenceClassification.from_pretrained(model_name)
print('model_done')

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

tokenizer_done
model_done


In [55]:
def compute_metrics(output):
  preds, labels = output
  preds = np.argmax(preds, axis=1) # numpy arrays
  # return accuracy.compute(predictions=preds, references=labels)
  return {"accuracy": (preds == labels).sum() / len(preds)}

def process_function(text):
  return tokenizer(
        text['text'],
        truncation=True
  )
  
# Функция для классификации текста
def classify_text(text):
    # Токенизация текста
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    
    # Получение предсказаний от модели
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Получаем вероятности классов
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=-1)
    
    # Получаем индекс класса с наибольшей вероятностью
    predicted_class = torch.argmax(probabilities, dim=-1).item()
    
    return predicted_class, probabilities


In [51]:

# Пример текста для классификации
text = ""

# Классификация текста
predicted_class, probabilities = classify_text(text)

print(f"Предсказанный класс: {predicted_class}")
print(f"Вероятности классов: {probabilities}")

Предсказанный класс: 0
Вероятности классов: tensor([[0.6354, 0.1477, 0.2170]])


In [56]:
dataset = load_dataset('MonoHime/ru_sentiment_dataset')

train = dataset['train']
test = dataset['validation']

train = train.map(process_function)
test = test.map(process_function)

Map: 100%|██████████| 21098/21098 [00:20<00:00, 1007.12 examples/s]


In [62]:
training_args = TrainingArguments(
  output_dir="C:/Стандартные папки/Рабочий стол/ответы на всош по ии/repo/cache/",
  learning_rate=3e-4,
  per_device_train_batch_size=64,
  per_device_eval_batch_size=8,
  num_train_epochs=10,
  weight_decay=0.01,
  evaluation_strategy="epoch",
  save_strategy="epoch",
  load_best_model_at_end=True,
  save_total_limit=3,

)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

FailedPreconditionError: C:/Стандартные папки/Рабочий стол/ответы на всош по ии/repo/cache is not a directory